In [8]:
# Dependencies
import pandas as pd
import numpy as np

In [9]:
# Read imdb dataset in csv format
movies_df = pd.read_csv('resources/IMDb_movies.csv', encoding='utf8')

C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# Filter columns to keep from dataset
cleaned_movies_df = movies_df.drop(columns=['date_published', 'description', 'original_title', 'duration', 'language', 'actors', 'avg_vote', 'votes', 'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore', 'reviews_from_users', 'reviews_from_critics'])
cleaned_movies_df.head()

,imdb_title_id,title,year,genre,country,director,writer,production_company
0,tt0000009,Miss Jerry,1894,Romance,USA,Alexander Black,Alexander Black,Alexander Black Photoplays
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,Charles Tait,Charles Tait,J. and N. Tait
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama
3,tt0002101,Cleopatra,1912,"Drama, History",USA,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film


In [11]:
# Read oscars awards dataset in csv format
oscars_df = pd.read_csv('resources/the_oscar_award.csv', encoding='utf8')
oscars_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [12]:
# Keep only a selected number of categories
cleaned_oscars_df = oscars_df[(oscars_df['winner']==True) &
                              ((oscars_df['category']=='BEST PICTURE') | 
                               (oscars_df['category']=='DIRECTING') | 
                               (oscars_df['category']=='BEST MOTION PICTURE') | 
                               (oscars_df['category']=='CINEMATOGRAPHY (Black-and-White)') |
                               (oscars_df['category']=='CINEMATOGRAPHY (Color)'))]
cleaned_oscars_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
59,1928,1929,2,DIRECTING,Frank Lloyd,The Divine Lady,True
98,1929,1930,3,DIRECTING,Lewis Milestone,All Quiet on the Western Front,True
139,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True
171,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True
234,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True


In [17]:
# Add movie ID to oscars movies dataframe
cleaned_oscars_df['title_id']=""

for index, film in cleaned_oscars_df.iterrows():
    for index2, title in cleaned_movies_df.iterrows():
            if (str(film['film']).lower() == str(title['title']).lower()):
                cleaned_oscars_df.loc[index, 'title_id'] = title['imdb_title_id']
cleaned_oscars_df.head()

C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,year_film,year_ceremony,ceremony,category,name,film,winner,title_id
59,1928,1929,2,DIRECTING,Frank Lloyd,The Divine Lady,True,
98,1929,1930,3,DIRECTING,Lewis Milestone,All Quiet on the Western Front,True,
139,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True,tt0163246
171,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True,tt3598108
234,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True,tt0427047


In [18]:
cleaned_movies_df.to_csv('output/cleaned_imdb.csv', index = False)
cleaned_oscars_df.to_csv('output/oscars_with_id.csv', index = False)

In [19]:
# Dropped rows where the awarded movies are not listed in the imdb dataset (empty rows)
cleaned2_oscars_df = cleaned_oscars_df.replace('', np.NaN)
cleaned2_oscars_df = cleaned2_oscars_df.dropna(how='any')

In [20]:
# Export oscars cleaned and filtered dataframe
cleaned2_oscars_df.to_csv('output/final_oscars.csv', index = False)

In [21]:
# Split director names in the director column in the imdb dataset into separates dataframe
directors_df = cleaned_movies_df['director'].str.split(',', n=5, expand = True)
director1_df = pd.DataFrame(directors_df[0])
director1_df = director1_df.rename(columns={0:'name'})
director2_df = pd.DataFrame(directors_df[1])
director2_df = director2_df.dropna(how='any')
director2_df = director2_df.rename(columns={1:'name'})

In [22]:
# Concat dataframes with all directors names
cleaned_directors = pd.concat([director1_df, director2_df], ignore_index = True)
cleaned_directors = pd.DataFrame(cleaned_directors['name'].unique())
cleaned_directors = cleaned_directors.rename(columns={0:'name'})

# Add director_id to directors
count=0
cleaned_directors['id']=""
for index, categories in cleaned_directors.iterrows():
    cleaned_directors.loc[index, 'id']=count
    count+=1
cleaned_directors.head()

,name,id
0,Alexander Black,0
1,Charles Tait,1
2,Urban Gad,2
3,Charles L. Gaskill,3
4,Francesco Bertolini,4


In [23]:
# Export directors dataframe to csv
cleaned_directors.to_csv('output/directors.csv', index = False)

In [24]:
# Oscars categories
cleaned_categories_df = pd.DataFrame(cleaned_oscars_df['category'].unique())
cleaned_categories_df = cleaned_categories_df.rename(columns={0:'categories'})

# Add oscars_id to categories
count=0
cleaned_categories_df['id']=""
for index, categories in cleaned_categories_df.iterrows():
    cleaned_categories_df.loc[index, 'id']=count
    count+=1
cleaned_categories_df

,categories,id
0,DIRECTING,0
1,CINEMATOGRAPHY (Black-and-White),1
2,CINEMATOGRAPHY (Color),2
3,BEST MOTION PICTURE,3
4,BEST PICTURE,4


In [25]:
# Export oscars categories
cleaned_categories_df.to_csv('output/oscars_categories.csv', index = False)

In [71]:
# # Oscars id per category
# directing_id = cleaned_categories_df[cleaned_categories_df['categories']=='DIRECTING']['id']
# cin_bw = cleaned_categories_df[cleaned_categories_df['categories']=='CINEMATOGRAPHY (Black-and-White)']['id']
# cin_color = cleaned_categories_df[cleaned_categories_df['categories']=='CINEMATOGRAPHY (Color)']['id']
# motion_pic = cleaned_categories_df[cleaned_categories_df['categories']=='BEST MOTION PICTURE']['id']
# best_pict = cleaned_categories_df[cleaned_categories_df['categories']=='BEST PICTURE']['id']

In [66]:
# from pandas import Series
# # Split directors names
# cleaned3_oscars_df = cleaned2_oscars_df.copy()

# for index, row in cleaned3_oscars_df.iterrows():
#     pd.concat([(row['category'], row['film'],  row['title_id'], row['oscar_id'], row['name'].split(','))])
    
# cleaned3_oscars_df

# cleaned3_oscars_df = cleaned3_oscars_df['name'].str.split(',', n=5, expand = True)

# for i in range(1, 5):
#     for row in cleaned3_oscars_df.iterrows():    
#         if str(row[i]) != 'None':
#             print(row[i])

# for index, row in cleaned3_oscars_df.iterrows():
    
    

# Split director names in the director column in the imdb dataset into separates dataframe
# directors_df = cleaned_movies_df['director'].str.split(',', n=5, expand = True)
# director1_df = pd.DataFrame(directors_df[0])
# director1_df = director1_df.rename(columns={0:'name'})
# director2_df = pd.DataFrame(directors_df[1])
# director2_df = director2_df.dropna(how='any')
# director2_df = director2_df.rename(columns={1:'name'})

In [67]:
# Assign Oscars id to their respective awarded movie title
cleaned2_oscars_df['oscar_id'] = ""

for index, movies in cleaned2_oscars_df.iterrows():
    if movies['category'] == 'DIRECTING':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 0
    elif movies['category'] == 'CINEMATOGRAPHY (Black-and-White)':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 1
    elif movies['category'] == 'CINEMATOGRAPHY (Color)':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 2
    elif movies['category'] == 'BEST MOTION PICTURE':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 3
    elif movies['category'] == 'BEST PICTURE':
        cleaned2_oscars_df.loc[index, 'oscar_id'] = 4

        
# Filter to only keep movie awards
movie_awards_df = cleaned2_oscars_df[
                              ((cleaned2_oscars_df['category']=='BEST PICTURE') | 
                               (cleaned2_oscars_df['category']=='BEST MOTION PICTURE') | 
                               (cleaned2_oscars_df['category']=='CINEMATOGRAPHY (Black-and-White)') |
                               (cleaned2_oscars_df['category']=='CINEMATOGRAPHY (Color)'))]
     
# Only keep title_id and oscar_id relationship dataset
movie_awards_df = movie_awards_df[['title_id', 'oscar_id']]

movie_awards_df.head()

,title_id,oscar_id
1678,tt0093351,1
1689,tt1781058,2
1840,tt0435669,1
2134,tt0026402,3
2243,tt0116477,3


In [68]:
# Export relationship of title_id with oscar_id
movie_awards_df.to_csv('output/movie_awards.csv', index = False)

In [69]:
# Filter to only keep directing awards (it reduces looping time by only looping over the directors that have won directing
# rather than filtering over the directors listed on the movie awards)
directing_awards_df = cleaned2_oscars_df[(cleaned2_oscars_df['category']=='DIRECTING')]

# Creating empty director_id column
directing_awards_df['director_id'] = ""

# Assign Director id to their respective awarded director name
for index, director in directing_awards_df.iterrows():
    for index2, reference in cleaned_directors.iterrows():
        if str(director['name']) == str(reference['name']):
            dir_id = cleaned_directors.at[index2, 'id']
            directing_awards_df.at[index, 'director_id'] = dir_id
#         else:
#             directing_awards_df.at[index, 'director_id'] = ""
# cleaned2_oscars_df




# Only keep title_id and oscar_id relationship dataset
directing_awards_df = directing_awards_df[['title_id', 'director_id']]

directing_awards_df.head()

C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title_id,director_id
139,tt0163246,386
171,tt3598108,106
234,tt0427047,68
360,tt0020025,61
1503,tt10497826,33


In [71]:
# Drop not found directors in the main imdb dataset
directing_awards_df = directing_awards_df.replace('', np.NaN)
directing_awards_df = directing_awards_df.dropna(how='any')
directing_awards_df.head(60)

,title_id,director_id
139,tt0163246,386.0
171,tt3598108,106.0
234,tt0427047,68.0
360,tt0020025,61.0
1503,tt10497826,33.0
2104,tt0026402,866.0
3404,tt0051658,779.0
3523,tt2638144,291.0
4010,tt0057590,1536.0
4133,tt0058385,346.0


In [121]:
cleaned2_oscars_df[cleaned2_oscars_df['category']=='DIRECTING']

,year_film,year_ceremony,ceremony,category,name,film,winner,title_id,oscar_id,director_id
139,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True,tt0163246,0,
171,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True,tt3598108,0,
234,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True,tt0427047,0,
360,1935,1936,8,DIRECTING,John Ford,The Informer,True,tt0020025,0,
1503,1943,1944,16,DIRECTING,Michael Curtiz,Casablanca,True,tt10497826,0,
2104,1947,1948,20,DIRECTING,Elia Kazan,Gentleman's Agreement,True,tt0026402,0,
3404,1958,1959,31,DIRECTING,Vincente Minnelli,Gigi,True,tt0051658,0,
3523,1959,1960,32,DIRECTING,William Wyler,Ben-Hur,True,tt2638144,0,
3768,1961,1962,34,DIRECTING,"Robert Wise, Jerome Robbins",West Side Story,True,tt0055614,0,
4010,1963,1964,36,DIRECTING,Tony Richardson,Tom Jones,True,tt0057590,0,


In [110]:
directing_awards_df.to_csv('output/director_awards.csv', index = False)